In [ ]:
import requests
from bs4 import BeautifulSoup

# URL to scrape
url = "https://shopeefood.vn/dong-nai/food/deals/1"

# Send a GET request
response = requests.get(url)

# Check if request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup)
    # Extract specific data (e.g., titles)
    titles = soup.find_all('h1')  # Adjust the tag to match the desired content
    for title in titles:
        print(title.text)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Set up the WebDriver (e.g., Chrome)
cService = webdriver.ChromeService(executable_path="/Users/ryanjones/Downloads/chromedriver-mac-arm64/chromedriver")  # Replace with your ChromeDriver path
driver = webdriver.Chrome(service=cService)

# Open the website
url = "https://shopeefood.vn/dong-nai/food/deals"  # Replace with the target URL
driver.get(url)

# Wait for the page to load (adjust the time if needed)
time.sleep(2)

# WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ul.pagination")))
# WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//li[@class="page-item '
#                                                                                 'active"]//a')))

# button = driver.find_element(By.CLASS_NAME, "icon icon-paging-next")
currentPageNumber = driver.find_element(By.XPATH, '//li[@class="active"]//a').text
a_elements = driver.find_elements(By.XPATH, "//div[@class='item-restaurant']/a")
hrefs=[a.get_attribute('href') for a in a_elements]
print(hrefs)



element = driver.find_element(By.XPATH, f"//*[text()='{int(currentPageNumber)+1}']")
element=element.find_element(By.XPATH, "parent::li")
driver.execute_script("arguments[0].click();", element)
# element.click()
# print(element)
# current_li = driver.find_element(By.XPATH, "//ul[@class='pagination']")
# button = current_li.find_element(By.CLASS_NAME, "active")
# button = button.find_element(By.XPATH, "preceding-sibling::li")

# button.click()
time.sleep(3)
a_elements = driver.find_elements(By.XPATH, "//div[@class='item-restaurant']/a")
hrefs=[a.get_attribute('href') for a in a_elements]
print(hrefs)

# Extract data (e.g., all titles with <h1> tag)
# titles = driver.find_elements(By.CLASS_NAME, "item-restaurant")

# Print the extracted titles
# for title in titles:
#     print(title.text)

# Close the browser
driver.quit()


In [127]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

with open('review_path.txt', 'r') as file:
    lines = file.readlines() 
txt="items.txt"
lines=[l.strip().split(" ")[1] for l in lines]

max_rep=5
cService = webdriver.ChromeService(executable_path="/Users/ryanjones/Downloads/chromedriver-mac-arm64/chromedriver")  # Replace with your ChromeDriver path
driver = webdriver.Chrome(service=cService)

start_at=106
for iln, ln in enumerate(lines[start_at:]):
    url = f"https://shopeefood.vn/dong-nai/highlands-coffee-tra-ca-phe-banh-396-nguyen-ai-quoc"  # Replace with the target URL
    driver.get(ln)
    # def wait_for_page_to_load():
    #     WebDriverWait(driver, 10).until(
    #         lambda driver: driver.execute_script("return document.readyState") == "complete"
    #     )

    # wait_for_page_to_load()

    time.sleep(1)
    try:
        kind=driver.find_element(By.CLASS_NAME, "kind-restaurant").text
        name=driver.find_element(By.CLASS_NAME, "name-restaurant").text
        addr=driver.find_element(By.CLASS_NAME, "address-restaurant").text
        otim=driver.find_element(By.CLASS_NAME, "time").text
    except:
        continue
    res_if=f"{iln+start_at}%{ln}%{kind}%{name}%{addr}%{otim}"

    scroll_position = driver.execute_script("""
        return {
            x: window.pageXOffset,
            y: window.pageYOffset
        };
    """)

    # Print the scroll position


    dow_to=500
    chk_it=[]
    to_write=[]
    old_off=0
    rep=0
    while True:
        driver.execute_script("window.scrollTo(0, window.pageYOffset+window.innerHeight*1.2);")
        time.sleep(0.5)
        scroll_position = driver.execute_script("""
            return {
                x: window.pageXOffset,
                y: window.pageYOffset
            };
        """)

        # Print the scroll position
        print(f"Current scroll position: x = {scroll_position['x']}, y = {scroll_position['y']}")

        if scroll_position['y']==old_off:
            rep+=1
        else:
            old_off=scroll_position['y']
            rep=0
        if rep>=max_rep:
            break
        
        try:
            rows=driver.find_elements(By.XPATH, "//div[@class='ReactVirtualized__Grid__innerScrollContainer']//div")
        
        except:
            continue
        cate=""

        for i, rw in enumerate(rows):
            try:
                r_tp=rw.get_attribute("class")
            except:
                continue
            if r_tp=="menu-group":
                cate=rw.text
            elif r_tp=="item-restaurant-row":
                try:
                    nm=rw.find_element(By.CLASS_NAME, "item-restaurant-name")
                    pr=rw.find_element(By.CLASS_NAME, "current-price")
                except:
                    continue
                if nm not in chk_it:
                    chk_it.append(nm)
                    
                    to_write.append(f"{res_if}&{cate}%{nm.text}%{pr.text}\n")
        
    
    act="a"
    if not os.path.exists(txt):
        act="w"
    with open(txt, act) as fi:
        for it in to_write:
            fi.write(it)
    
        # current_scroll_position = driver.execute_script("return window.pageYOffset")
        # document_height = driver.execute_script("return document.body.scrollHeight")
        # viewport_height = driver.execute_script("return window.innerHeight")



Current scroll position: x = 0, y = 2503
Current scroll position: x = 0, y = 2503
Current scroll position: x = 0, y = 2503
Current scroll position: x = 0, y = 2503
Current scroll position: x = 0, y = 768
Current scroll position: x = 0, y = 1536
Current scroll position: x = 0, y = 2304
Current scroll position: x = 0, y = 3072
Current scroll position: x = 0, y = 3840
Current scroll position: x = 0, y = 4608
Current scroll position: x = 0, y = 5376
Current scroll position: x = 0, y = 6144
Current scroll position: x = 0, y = 6750
Current scroll position: x = 0, y = 6750
Current scroll position: x = 0, y = 6750
Current scroll position: x = 0, y = 6750
Current scroll position: x = 0, y = 6750
Current scroll position: x = 0, y = 6750
Current scroll position: x = 0, y = 768
Current scroll position: x = 0, y = 1536
Current scroll position: x = 0, y = 2304
Current scroll position: x = 0, y = 3072
Current scroll position: x = 0, y = 3840
Current scroll position: x = 0, y = 4608
Current scroll pos

KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os

# Set up the WebDriver (e.g., Chrome)
cService = webdriver.ChromeService(executable_path="/Users/ryanjones/Downloads/chromedriver-mac-arm64/chromedriver")  # Replace with your ChromeDriver path
driver = webdriver.Chrome(service=cService)

locations=[
            "ho-chi-minh", "ha-noi", "da-nang", "can-tho", "hai-phong", "hue", "khanh-hoa", \
            "dong-nai", "nghe-an", "vung-tau", 
            "an-giang", "bac-lieu", "bac-ninh", "ben-tre", \
            "binh-duong", "binh-dinh", "binh-phuoc", "binh-thuan", "ca-mau", "dak-lak", "dien-bien", \
            "hai-duong", "dong-thap", "gia-lai", "hau-giang", "kien-giang", "lam-dong", "long-an", "nam-dinh", \
            "ninh-thuan", "phu-yen", "quang-nam", "quang-ninh", "soc-trang", "tay-ninh", "thai-nguyen", "thanh-hoa", "tien-giang",\
            "tra-vinh", "vinh-long"
            ]

txt="review_path.txt"
i_lc=0
max_page=40
load=True
while True:
    if load:
        lc=locations[i_lc]
        url = f"https://shopeefood.vn/{lc}/food/deals"  # Replace with the target URL
        driver.get(url)
    try:
        time.sleep(2)
        

        hrefs = driver.find_elements(By.XPATH, "//div[@class='item-restaurant']/a")
        hrefs=[a.get_attribute('href') for a in hrefs]

        cur_pag=driver.find_element(By.XPATH, '//li[@class="active"]//a')

        act="a"
        if not os.path.exists(txt):
            act="w"
        cur_pag_num=cur_pag.text if cur_pag else 1
        with open(txt, act) as fi:
            for hr in hrefs:
                fi.write(f"{cur_pag_num} {hr}\n")
        
        print("ASDFA")
        nex_pag=None
        if cur_pag:
            nex_pag=driver.find_element(By.XPATH, f"//*[text()='{int(cur_pag.text)+1}']")
            if nex_pag:
                nex_pag=nex_pag.find_element(By.XPATH, "parent::li")
            else:
                nex_pag=None
        
        if nex_pag:
            if int(nex_pag.text)>max_page:
                i_lc+=1
                load=True
            else:
                driver.execute_script("arguments[0].click();", nex_pag)
                load=False
        else:
            i_lc+=1
            load=True


    except Exception as e:
        err=f"error with {lc} {e}"
        print(f"error with {lc} {e}")
        if '{"method":"xpath","selector":"//li[@class="active"]//a"' in err:
            act="a"
            if not os.path.exists(txt):
                act="w"
            with open(txt, act) as fi:
                for hr in hrefs:
                    fi.write(f"{1} {hr}\n")
            
            i_lc+=1
            load=True
        elif '//*[text()=' in err:
            i_lc+=1
            load=True
        else:
            driver.quit()
            break
